In [14]:
import numpy as np
import pandas as pd
from src import load_data

In [58]:
summary = load_data.get_summary_data("data/raw/summary.xlsx")
summary

,Duration,Sends,Opens,Clicks,Open Rate,Click Rate,Unsubscribes,Word Count,Link Count,Month,Day,Year,Week,Weekday
Date/Time,,,,,,,,,,,,,,
2020-07-20 06:47:17.764,0:00:25.135,87,102,6,1.1724,0.0588,0,450,21,7,20,2020,30,0
2020-07-21 06:33:50.176,0:00:33.135,74,142,22,1.9189,0.1549,0,478,21,7,21,2020,30,1
2020-07-22 06:28:15.271,0:00:33.872,78,123,41,1.5769,0.3333,0,496,21,7,22,2020,30,2
2020-07-23 06:38:34.308,0:00:33.712,82,135,7,1.6463,0.0519,0,496,24,7,23,2020,30,3
2020-07-24 08:19:54.483,0:00:35.835,83,270,15,3.2530,0.0556,0,515,18,7,24,2020,30,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-10-01 09:34:55.009,0:21:23.488,3648,1588,245,0.4353,0.1543,1,536,45,10,1,2021,39,4
2021-10-04 07:47:17.811,0:15:51.485,3674,1978,307,0.5384,0.1552,2,449,42,10,4,2021,40,0
2021-10-06 08:41:22.988,0:18:06.223,3718,1748,333,0.4701,0.1905,4,499,49,10,6,2021,40,2


In [13]:
link_click_data = pd.read_csv("data/processed/Link Click Data.csv", parse_dates=True, index_col="Date")
link_click_data.drop(labels="Unnamed: 0", axis=1, inplace=True)
link_click_data.head()

,ArticleNumber,LinkText,Tag,Link,Clicks,Month,Day,Year,Week,Weekday
Date,,,,,,,,,,
2020-07-20,0,Celanese,3,https://en.wikipedia.org/wiki/Celanese,0.0,7,20,2020,30,0
2020-07-20,0,Clariant,4,https://en.wikipedia.org/wiki/Clariant,0.0,7,20,2020,30,0
2020-07-20,0,Orbia,5,https://en.wikipedia.org/wiki/Orbia,0.0,7,20,2020,30,0
2020-07-20,0,Westlake Chemical,6,https://en.wikipedia.org/wiki/Westlake_Chemical,1.0,7,20,2020,30,0
2020-07-20,0,by steam cracking natural gas,7,https://en.wikipedia.org/wiki/Ethylene#Industr...,0.0,7,20,2020,30,0


In [57]:
min_to_max = summary.describe().transpose()[["min","25%","50%","75%","max"]][0:8]
min_to_max.reset_index(inplace=True)
min_to_max.rename(columns={"index":"Measure"}, inplace=True)
data = min_to_max.to_dict("list")
columns = min_to_max.columns
data

{'Measure': ['Sends',
  'Opens',
  'Clicks',
  'Open Rate',
  'Click Rate',
  'Unsubscribes',
  'Word Count',
  'Link Count'],
 'min': [61.0, 102.0, 6.0, 0.3852, 0.0366, 0.0, 355.0, 18.0],
 '25%': [827.0, 555.0, 77.0, 0.5145, 0.1337, 0.0, 463.0, 29.0],
 '50%': [1380.0, 819.0, 214.0, 0.5705, 0.1947, 1.0, 489.0, 40.0],
 '75%': [2494.0, 1277.0, 337.0, 0.6378, 0.3135, 2.0, 513.0, 54.0],
 'max': [3782.0, 1978.0, 1331.0, 3.253, 0.9562, 9.0, 621.0, 79.0]}

In [89]:
article_section_data = pd.read_csv("data/processed/Article Section Data.csv")
article_section_data.drop(labels=["Unnamed: 0"], axis = 1, inplace=True)
#article_section_data["Weekday"] = (article_section_data["Date"].astype(dtype=np.datetime64)).dt.weekday

In [103]:
article_section_data.groupby(["Weekday","ArticleNumber"]).sum().reset_index().query("ArticleNumber == 0")#.pivot_table(values = "LinkCount", columns=["ArticleNumber"],index=["Weekday"]).mean().dropna()

,Weekday,ArticleNumber,LinkCount,SectionArticleLength,Clicks
0,0,0,524,59434,6259.0
5,1,0,167,21634,748.0
8,2,0,573,59406,5295.0
13,3,0,154,20693,836.0
16,4,0,546,59100,5612.0
